# Regresión con Scikit-learn

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />

## 1. Regresión

En regresión, la etiqueta es continua, es decir una salida real. Por ejemplo, en astronomía, la tarea de determinar si un objeto es una estrella, una galaxia o un cuásar es un problema de clasificación: la etiqueta viene de tres categorías distintas. Por otro lado, podríamos querer estimar la edad de un objeto basándonos en su imagen: esto sería regresión, porque la etiqueta (edad) es una cantidad continua.

Usaremos el dataet diabetes de scikit-learn, el cual consiste de 10 variables fisiológicas (age, sex, weight, blood pressure) medidas en 442 pacientes, y el avance de la enfermedad después de un año.

- age: Edad
- sex: Sexo
- body mass index: Índice de masa corporal
- average blood pressure: Presión sanguinea promedio
- s1-s6: Mediciones del suero sanguíneo

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets

diabetes = datasets.load_diabetes()
diabetes.keys()

In [ ]:
print(diabetes['DESCR'])

In [ ]:
# Creamos un dataframe a partir del dataset cargado desde scikit-learn

diabetes_df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
diabetes_df["avance"]=diabetes.target

print("Tamaño del dataset: ", len(diabetes_df))
diabetes_df.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from pandas.plotting import scatter_matrix

plot=scatter_matrix(diabetes_df, alpha=0.2, figsize=[15, 15], marker=".", diagonal='kde')

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(diabetes_df, test_size=0.3)

print("Tamaño del train set: ", len(df_train))
print("Tamaño del test set: ", len(df_test))

# 2. Regresión lineal

$$ \text{min}_{\mathbf{w}, b} \sum_i || \mathbf{w}^\mathsf{T}\mathbf{x}_i + b  - y_i||^2 $$

## 2.1 Caso simple (1 feature)

Partiremos consturyendo un modelo lineal usando el campo bmi, el cual corresponde a la columna numero 2

Usaremos la función LinearRegression que permite crear un modelo lineal usando la regresión de mínimos cuadrados.

In [ ]:
X_train=np.array(df_train.iloc[:,2])[np.newaxis].T
y_train=np.array(df_train.iloc[:,10])

X_test=np.array(df_test.iloc[:,2])[np.newaxis].T
y_test=np.array(df_test.iloc[:,10])

In [ ]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)

print("R^2 en entrenamiento: %f" % regr.score(X_train, y_train))
print("R^2 en test: %f" % regr.score(X_test, y_test))

In [ ]:
print(regr.coef_)

In [ ]:
y_pred = regr.predict(X_test)

In [ ]:
# Usaremos el campo bmi para graficar el ajuste lineal

plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xlabel('Indice de masa corporal')
plt.ylabel('Avance enfermedad')

plt.show()

## 2.2 Caso multivariado (Todos los features)

Usaremos todos los features para generar un modelo lineal

In [ ]:
X_train=np.array(df_train.iloc[:,0:10])
y_train=np.array(df_train.iloc[:,10])

X_test=np.array(df_test.iloc[:,0:10])
y_test=np.array(df_test.iloc[:,10])

In [ ]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train, y_train)

print("R^2 en entrenamiento: %f" % regr.score(X_train, y_train))
print("R^2 en test: %f" % regr.score(X_test, y_test))

In [ ]:
print(regr.coef_)

## 2.3 Construcción de curva de aprendizaje

Scikit-learn permite calcular la curva de aprendizaje usando la función learning_curve. Es una herramienta que permite determinar cuanto ganamos al agregar mas datos al set de entrenamiento y determinar si el estimador sufre de un error de varianza o un erro de bias.

In [ ]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(est, X, y):
    training_set_size, train_scores, test_scores = learning_curve(est, X, y, train_sizes=np.linspace(.1, 1, 20))
    estimator_name = est.__class__.__name__
    line = plt.plot(training_set_size, train_scores.mean(axis=1), '--', label="puntuaciones de entrenamiento " + estimator_name)
    plt.plot(training_set_size, test_scores.mean(axis=1), '-', label="puntuaciones de test " + estimator_name, c=line[0].get_color())
    plt.xlabel('Tamaño del conjunto de entrenamiento')
    plt.legend(loc='best')
    plt.ylim(-0.1, 1.1)


In [ ]:
X=np.array(diabetes_df.iloc[:,0:10])
y=np.array(diabetes_df.iloc[:,10])

plt.figure()
plot_learning_curve(LinearRegression(), X_train, y_train)

# 3. Regresión de cresta (*Ridge Regression*, penalización L2)

**El estimador de cresta (``Ridge``)** es una regularización simple (llamada regularización L2) para el modelo LinearRegression. En particular, tiene el beneficio de no ser más costoso computacionalmente que la estimación basada en mínimos cuadrados.

$$ \text{min}_{\mathbf{w},b}  \sum_i || \mathbf{w}^\mathsf{T}\mathbf{x}_i + b  - y_i||^2  + \alpha ||\mathbf{w}||_2^2$$ 

In [ ]:
from sklearn.linear_model import Ridge
ridge_models = {}
training_scores = []
test_scores = []

for alpha in [100, 10, 1, .01]:
    ridge = Ridge(alpha=alpha).fit(X_train, y_train)
    training_scores.append(ridge.score(X_train, y_train))
    test_scores.append(ridge.score(X_test, y_test))
    ridge_models[alpha] = ridge

plt.figure()
plt.plot(training_scores, label="puntuaciones de entrenamiento")
plt.plot(test_scores, label="puntuaciones de test")
plt.xticks(range(4), [100, 10, 1, .01])
plt.legend(loc="best")

La cantidad de regularización se ajusta a través del parámetro `alpha` del modelo Ridge.

In [ ]:
plt.figure()
plot_learning_curve(LinearRegression(), X, y)
plot_learning_curve(Ridge(alpha=10), X, y)

In [ ]:
alphas = np.logspace(-4, -1, 6)
print("Alphas: ", alphas)

ridge = Ridge(alpha=.1)
scores = [ridge.set_params(alpha=alpha).fit(X_train, y_train,).score(X_test, y_test) for alpha in alphas]
print("Score: ", scores) 

best_alpha = alphas[scores.index(max(scores))]
print("Best alpha: ", best_alpha)

# 4. Lasso (penalización L1)
**El estimador ``Lasso``** es útil para conseguir imponer dispersión en los coeficientes. En otras palabras, se debería preferir esta penalización si creemos que muchas de las características no son relevantes. Se consigue a través de la regularización L1.

$$ \text{min}_{\mathbf{w}, b} \sum_i \frac{1}{2} || \mathbf{w}^\mathsf{T}\mathbf{x}_i + b  - y_i||^2  + \alpha ||\mathbf{w}||_1$$ 

Del primer gráfico podemos ver que el campo sex no aporta mucha información en la predicción de la variable avance de enfermedad.

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
scores = [lasso.set_params(alpha=alpha).fit(X_train, y_train).score(X_test, y_test) for alpha in alphas]
best_alpha = alphas[scores.index(max(scores))]

print("Best alpha: ", best_alpha)

lasso.alpha = best_alpha
lasso.fit(X_train, y_train)

In [ ]:
print(lasso.coef_)